In [1]:
import requests
import bs4
import pandas as pd

### Physical Description

In [65]:
url = "https://beadage.net/gemstones/"
r = requests.get(url)
soup = bs4.BeautifulSoup(r.content, 'html.parser')
aa = soup.find_all('a', {'class': "term-link"}) 

In [66]:
# NOTE: 136 requests will take a while
l = []
for a in aa:
    rr = requests.get(a.get('href'))
    ssoup = bs4.BeautifulSoup(rr.content, 'html.parser')
    h2s = ssoup.find_all('h2')
    description = ""
    for h2 in h2s:
        if h2.text.strip().lower().endswith("gemstone"):
            next_element = h2.find_next_sibling()
            while next_element and next_element.name != 'h2':
                if next_element.name == 'p':
                    description = description + next_element.text.replace('\xa0', ' ') + "\n"
                next_element = next_element.find_next_sibling()
    l.append([a.text.strip().lower(), description])

In [83]:
# Export csv with all stones and physical descriptions
d = pd.DataFrame(l, columns=["stone", "physical_description"])
d = d[d.physical_description != ""].drop_duplicates(subset='stone').reset_index(drop=True)
fname = "data/output/phys_desc.csv"
d.to_csv(fname, index=False)

### Meanings

In [2]:
# Scrape descriptions
url = 'https://beadage.net/gemstones/'
r = requests.get(url)
soup = bs4.BeautifulSoup(r.content, 'html.parser')
a = soup.find('div', {'class': "results"})
b = a.find_all('div', {'class': "card"})
l = []
for stew in b:
    c = stew.find_all('a')[1]
    stone = c.get('title')
    link = c.get('href')
    description = stew.find('p').text
    l.append([stone, link, description])
meanings = pd.DataFrame(l, columns=["stone", "link", "description"])

In [3]:
meanings.stone = meanings.stone.str.lower()

In [4]:
print(meanings.shape)
meanings.head()

(100, 3)


,stone,link,description
0,agate,https://beadage.net/gemstones/agate/,"Agates are earth stones. Overall, agates are g..."
1,alexandrite,https://beadage.net/gemstones/alexandrite/,"Rather than just bring you joy, Alexandrite he..."
2,amazonite,https://beadage.net/gemstones/amazonite/,Amazonite helps to harmonize apparently differ...
3,amber,https://beadage.net/gemstones/amber/,"Amber carries the energy of the sun, warmth, a..."
4,amethyst,https://beadage.net/gemstones/amethyst/,Amethyst is a powerhouse stone. It vibrates at...


In [20]:
# Export csv
fname = "data/output/meanings.csv"
meanings.to_csv(fname, index=False)

### Hardness

In [8]:
r = requests.get("https://www.gemsociety.org/article/select-gems-ordered-mohs-hardness/")
soup = bs4.BeautifulSoup(r.content, 'html.parser')

In [9]:
def convHardness(s):
    n = 0
    # split ranges
    if '-' in s:
        n1 = 0
        n2 = 0
        a = s.replace(" ", "").split("-")
        if a[0][-1] == "½":
            n1 += 0.5
        n1 += int(a[0].replace("½", ""))
        if a[1][-1] == "½":
            n2 += 0.5
        n2 += int(a[1].replace("½", ""))
        n = (n1 + n2) / 2
        return n
    if s[-1] == "½":
        n = 0.5
    n += int(s.replace("½", ""))
    return n

In [10]:
a = soup.find_all('tbody')[1].find_all('tr')
l = []
for rw in a:
    e = rw.find_all('td')
    if e[0].text != "":
        l.append([e[0].text, convHardness(e[1].text)])
    l.append([e[2].text, convHardness(e[3].text)])
hardness = pd.DataFrame(l, columns=["stone", "hardness"])

In [16]:
hardness.sort_values('hardness', ascending=False, inplace=True)
hardness.reset_index(drop=True, inplace=True)
hardness.stone = hardness.stone.str.lower()

In [17]:
print(hardness.shape)
hardness.head()

(346, 2)


,stone,hardness
0,diamond,10.0
1,synth. moissanit,9.5
2,sapphire,9.0
3,ruby,9.0
4,yag,8.5


In [21]:
# Export csv
fname = "data/output/hardness.csv"
hardness.to_csv(fname, index=False)

### Chakra, Zodiac, Birthstone, Element

In [64]:
stones = ["Agate", "Aventurine", "Aquamarine", "Apatite", "Amethyst", "Amazonite", "Bloodstone", 
         "chrysoprase-healing-properties", "Citrine", "crystal-quartz-properties", "dumortierite", "fluorite"]
url = "https://injewels.net/blogs/healing-properties-meanings/"
ls = []
stone = "Apache Tears"
for stone in stones:
    r = requests.get(url + stone)
    soup = bs4.BeautifulSoup(r.content, 'html.parser')
    a = soup.find_all('script', {'type': "application/ld+json"})
    for p in a:
        if "Symbolism:\\nChakra:" in p.contents[0]:
            # we found the right p
            ss = p.contents[0]
            i = ss.find("Symbolism:\\nChakra:")
            j = ss.find("\\nZodiac:")
            chak = ss[i+20:j]
            k = ss.find("\\nBirthstone")
            zodiac = ss[j+10:k]
            l = ss.find("\\nElement")
            birthstone = ss[k+14:l]
            element = ss[l+11:l + ss[l+1:].find("\\n") + 1]
            ls.append((stone.lower(), chak, zodiac, birthstone, element))

In [65]:
data2 = pd.DataFrame(ls, columns=["stone", "chakra", "zodiac", "birthstone", "element"])

In [66]:
data2.iloc[7,0] = "chrysoprase"
data2.iloc[9,0] = "crystal quartz"

In [67]:
df2 = data2.merge(df, on='stone', how='outer')

In [68]:
df.stone[:25]

0                 agate
1           alexandrite
2             amazonite
3                 amber
4              amethyst
5              ametrine
6     angel aura quartz
7              angelite
8          apache tears
9               apatite
10           aquamarine
11            aragonite
12           aventurine
13              azurite
14     black tourmaline
15           bloodstone
16      blue chalcedony
17      blue lace agate
18              calcite
19            carnelian
20            celestite
21             charoite
22          chrysocolla
23          chrysoprase
24              citrine
Name: stone, dtype: object

In [76]:
df[df.stone.str.contains('quartz')]

,stone,link,description,hardness,stone_class
6,angel aura quartz,https://beadage.net/gemstones/angel-aura-quartz/,Angel Aura Quartz is an enhanced version of cl...,NaN,quartz
68,quartz,https://beadage.net/gemstones/quartz/,Clear quartz crystal is the most useful and ve...,7.0,quartz
74,rose quartz,https://beadage.net/gemstones/rose-quartz/,Rose quartz is the classic stone of love. It h...,NaN,quartz
77,rutilated quartz,https://beadage.net/gemstones/rutilated-quartz/,"Rutilated quartz acts as a two-way antenna, am...",NaN,quartz
84,smoky quartz,https://beadage.net/gemstones/smoky-quartz/,Smoky quartz clears negative energies from the...,7.0,quartz


In [100]:
p = 'apatite'
df[df.stone==p].index[0]

9

In [104]:
df.iloc[9, 4]

'apatite'

In [51]:
df['stone_class'] = [i.split()[-1] for i in df.stone]

In [105]:
#df['waterproof']
#chakra_data = pd.read_csv('', sep=",", header=True)
df['chakra'] = 
#df.stone.map(chakra_data)[:25]

AttributeError: module 'pandas' has no attribute 'nan'

In [52]:
df[:10]

,stone,link,description,hardness,stone_class
0,agate,https://beadage.net/gemstones/agate/,"Agates are earth stones. Overall, agates are g...",NaN,agate
1,alexandrite,https://beadage.net/gemstones/alexandrite/,"Rather than just bring you joy, Alexandrite he...",8.50,alexandrite
2,amazonite,https://beadage.net/gemstones/amazonite/,Amazonite helps to harmonize apparently differ...,6.25,amazonite
3,amber,https://beadage.net/gemstones/amber/,"Amber carries the energy of the sun, warmth, a...",2.25,amber
4,amethyst,https://beadage.net/gemstones/amethyst/,Amethyst is a powerhouse stone. It vibrates at...,7.00,amethyst
5,ametrine,https://beadage.net/gemstones/ametrine/,Ametrine combines the healing properties of Am...,NaN,ametrine
6,angel aura quartz,https://beadage.net/gemstones/angel-aura-quartz/,Angel Aura Quartz is an enhanced version of cl...,NaN,quartz
7,angelite,https://beadage.net/gemstones/angelite/,Angelite helps with communication with your an...,NaN,angelite
8,apache tears,https://beadage.net/gemstones/apache-tears/,Apache Tears are helpful for surfacing grief h...,NaN,tears
9,apatite,https://beadage.net/gemstones/apatite/,Green apatite helps sooth the nervous system a...,5.00,apatite


In [73]:
df.shape

(100, 5)

In [54]:
g = df.groupby("stone_class")

In [55]:
g.count().sort_values(['stone', 'hardness'], ascending=[False, False])

,stone,link,description,hardness
stone_class,,,,
quartz,5,5,5,2
jasper,5,5,5,1
obsidian,5,5,5,0
tourmaline,5,5,5,0
agate,3,3,3,0
...,...,...,...,...
sunstone,1,1,1,0
tears,1,1,1,0
unakite,1,1,1,0


In [56]:
subcats = g.mean()

In [57]:
subcats['count'] = g.count().stone

In [58]:
subcats.sort_values('count', ascending=False)

,hardness,count
stone_class,,
tourmaline,NaN,5
quartz,7.00,5
jasper,6.75,5
obsidian,NaN,5
agate,NaN,3
...,...,...
garnet,7.00,1
fluorite,NaN,1
eye,NaN,1


In [59]:
df[df.stone_class == "quartz"]

,stone,link,description,hardness,stone_class
6,angel aura quartz,https://beadage.net/gemstones/angel-aura-quartz/,Angel Aura Quartz is an enhanced version of cl...,NaN,quartz
68,quartz,https://beadage.net/gemstones/quartz/,Clear quartz crystal is the most useful and ve...,7.0,quartz
74,rose quartz,https://beadage.net/gemstones/rose-quartz/,Rose quartz is the classic stone of love. It h...,NaN,quartz
77,rutilated quartz,https://beadage.net/gemstones/rutilated-quartz/,"Rutilated quartz acts as a two-way antenna, am...",NaN,quartz
84,smoky quartz,https://beadage.net/gemstones/smoky-quartz/,Smoky quartz clears negative energies from the...,7.0,quartz


In [74]:
df2[:15]

,stone,chakra,zodiac,birthstone,element,link,description,hardness,stone_class
0,agate,Third Eye and Crown,Virgo and Gemini,September,Earth,https://beadage.net/gemstones/agate/,"Agates are earth stones. Overall, agates are g...",NaN,agate
1,aventurine,Heart,"Libra, Virgo",August,Earth,https://beadage.net/gemstones/aventurine/,Green aventurine is known as the “Lucky Talism...,7.00,aventurine
2,aquamarine,Throat,Scorpio,March,Water,https://beadage.net/gemstones/aquamarine/,Aquamarine cools anger and helps create a calm...,7.75,aquamarine
3,apatite,Throat,Gemini,December,Air,https://beadage.net/gemstones/apatite/,Green apatite helps sooth the nervous system a...,5.00,apatite
4,amethyst,"Third Eye, Crown","Virgo, Sagittarius, Capricorn, Aquarius, Pisces",February,"Water, Air",https://beadage.net/gemstones/amethyst/,Amethyst is a powerhouse stone. It vibrates at...,7.00,amethyst
5,amazonite,Heart and Throat,Virgo,September,Earth and Water,https://beadage.net/gemstones/amazonite/,Amazonite helps to harmonize apparently differ...,6.25,amazonite
6,bloodstone,Root,"Aries, Libra, Pisces",March,Earth,https://beadage.net/gemstones/bloodstone/,Bloodstone assists us in facing the hard reali...,NaN,bloodstone
7,chrysoprase,Heart,"Taurus, Gemini + Libra",May,Earth,https://beadage.net/gemstones/chrysoprase/,Chrysoprase reminds and reassure you of your h...,6.75,chrysoprase
8,citrine,"Sacral, Solar Plexus + Root","Aries, Gemini, Leo, Libra",November,Fire,https://beadage.net/gemstones/citrine/,"To manifest your dreams, you first need to kno...",7.00,citrine
9,crystal quartz,All,"Leo, Virgo, Aries",April,Earth,NaN,NaN,NaN,NaN


### Astrology

In [1]:
from client import ApiClient

In [ ]:
client = ApiClient('ef969b99-85cd-41e7-8d4c-1d239cf37601', 'BHwBpgS6LnHL1x5xDxWwTnymQ5qUHNvz5w7wa1tU')
result = client.get('v2/astrology/kundli/advanced', {
    'ayanamsa': 1,
    'coordinates': '41.497903369,-81.709036073',
    'datetime': '1996-06-19T02:26:30+00:00'
})
data = result.get('data')
# Warning: Limited times I can run this

In [ ]:
data.get('nakshatra_details')
# TODO